In [ ]:
!apt install -y espeak-ng
!pip install torch torchaudio uv
!git clone https://github.com/Zyphra/Zonos.git
%cd Zonos
!pip install -e .

In [ ]:
!pip install --upgrade numpy


In [ ]:
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
from zonos.utils import DEFAULT_DEVICE as device

model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

In [ ]:
import re

text = '''0011_000283	They had been named Tom and Jerry.	Neutral
0011_000284	Chapter eighteen, the return journey.	Neutral
0011_000285	The straw, charcoal and the pea.	Neutral
0011_000286	One two three, where is your breakfast.	Neutral
0011_000287	A voice said: Come in.	Neutral'''

parsed_lines = []
for line in text.split("\n"):
    match = re.match(r"^\S+\t(.*?)\tNeutral$", line)
    if match:
        parsed_lines.append(match.group(1))

print(parsed_lines)


['They had been named Tom and Jerry.', 'Chapter eighteen, the return journey.', 'The straw, charcoal and the pea.', 'One two three, where is your breakfast.', 'A voice said: Come in.']


In [ ]:
import torchaudio
import torch

wav, sampling_rate = torchaudio.load('/content/0011_merged.wav')
speaker = model.make_speaker_embedding(wav, sampling_rate)

emotions = {
    "happy": [1, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077],
    "sad": [0.3077, 1, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077], 
    "angry": [0.3, 0.02, 0.02, 0.02, 0.02, 1, 0.2, 0.3],
    "surprised": [0.3077, 0.0256, 0.0256, 0.0256, 1, 0.0256, 0.2564, 0.3077],
    "neutral": [0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077]
}

for num, text in enumerate(parsed_lines):
  for emotion_name, emotion_vector in emotions.items():
      cond_dict = make_cond_dict(
          text=text,
          speaker=speaker,
          emotion=emotion_vector
      )

      # Генерируем голос
      conditioning = model.prepare_conditioning(cond_dict)
      codes = model.generate(conditioning)
      wavs = model.autoencoder.decode(codes).cpu()

      torchaudio.save(f'/content/drive/MyDrive/dataset/0011/{emotion_name}/{num + 283}.wav', wavs[0], model.autoencoder.sampling_rate)


ResNet293_SimAM_ASP_base.pt:   0%|          | 0.00/397M [00:00<?, ?B/s]

ResNet293_SimAM_ASP_base_LDA-128.pt:   0%|          | 0.00/265k [00:00<?, ?B/s]

Generating:   0%|          | 0/2588 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
W0324 21:41:45.349000 2740 torch/_inductor/utils.py:1137] [3/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipp